In [41]:
import jams
import numpy as np

diccionario_tonicas = { 
    'C': 0,
    'C#': 1,
    'D': 2,
    'D#': 3,
    'E': 4,
    'F': 5,
    'F#': 6,
    'G': 7,
    'G#': 8,
    'A': 9,
    'A#': 10,
    'B': 11
}
diccionario_acordes = {
    'maj': [0, 4, 7],
    'min': [0, 3, 7],
    '7': [0, 4, 7, 10],
    'maj7': [0, 4, 7, 11],
    'min7': [0, 3, 7, 10],
    'dim': [0, 3, 6],
    'dim7': [0, 3, 6, 9],
    'aug': [0, 4, 8],
    'aug7': [0, 4, 8, 10],
    'sus2': [0, 2, 7],
    'sus4': [0, 5, 7],
    '7sus4': [0, 5, 7, 10],
    '6': [0, 4, 7, 9],
    'm6': [0, 3, 7, 9],
    '9': [0, 4, 7, 10, 14],
    'maj9': [0, 4, 7, 11, 14],
    'min9': [0, 3, 7, 10, 14],
    '11': [0, 4, 7, 10, 14, 17],
    'maj11': [0, 4, 7, 11, 14, 17],
    'min11': [0, 3, 7, 10, 14, 17],
    '13': [0, 4, 7, 10, 14, 17, 21],
    'maj13': [0, 4, 7, 11, 14, 17, 21],
    'min13': [0, 3, 7, 10, 14, 17, 21],
    '7#9': [0, 4, 7, 10, 15],
    '7b9': [0, 4, 7, 10, 13],
    '7#5': [0, 4, 8, 10],
    '7b5': [0, 4, 6, 10],
    'hdim7': [0, 3, 6, 10],
    }

# Función para encontrar la duración permitida más cercana a una duración dada en segundos
def encontrar_duracion_permitida(duracion, duraciones_permitidas_seg):
    duracion_permitida_cercana = min(duraciones_permitidas_seg.values(), key=lambda x: abs(x - duracion))
    return duracion_permitida_cercana
# Función para representar una duración como un vector binario
def duracion_a_vec(duracion, duraciones_permitidas_seg):
    duracion_permitida_cercana = encontrar_duracion_permitida(duracion, duraciones_permitidas_seg)
    vector_binario = np.zeros(len(duraciones_permitidas_seg))
    for i, duracion_permitida in enumerate(duraciones_permitidas_seg.values()):
        if duracion_permitida == duracion_permitida_cercana:
            vector_binario[i] = 1
    return vector_binario

# Función para representar un desfase como un vector binario
def offset_a_vec(offset, offsets_permitidos_seg):
    # Buscar el desfase permitido más cercano
    offset_permitido_cercano = min(offsets_permitidos_seg, key=lambda x:abs(x-offset))
    vector_binario = np.zeros(len(offsets_permitidos_seg))
    for i, offset_permitido in enumerate(offsets_permitidos_seg):
        if offset_permitido == offset_permitido_cercano:
            vector_binario[i] = 1
    return vector_binario
def acorde_a_vec(acorde):
    acorde_vec = np.zeros(12)
    tonica = acorde.value.split(':')[0]
    tipo_acorde = acorde.value.split(':')[1]
    idx_tonica = diccionario_tonicas[tonica]
    idx_tipo_acorde = diccionario_acordes[tipo_acorde]
    for idx in idx_tipo_acorde:
        acorde_vec[(idx_tonica + idx) % 12] = 1    
    return acorde_vec
# Función para dividir un silencio largo en duraciones permitidas
def dividir_silencio(delta_t,max_dur,duraciones_permitidas_seg): 
    duraciones_divididas = []
    while delta_t > max_dur:
        duracion_permitida = encontrar_duracion_permitida(delta_t, duraciones_permitidas_seg)
        duraciones_divididas.append(duracion_permitida)
        delta_t -= duracion_permitida
    if delta_t <= max_dur:
        duracion_permitida = encontrar_duracion_permitida(delta_t, duraciones_permitidas_seg)
        duraciones_divididas.append(duracion_permitida)
        
    return duraciones_divididas


def jams_a_vec(ruta_archivo_jams):
    # Cargar el archivo JAMS
    jam = jams.load(ruta_archivo_jams)

    # Buscar anotaciones de notas MIDI
    notes = jam.search(namespace='note_midi')
    chords = jam.search(namespace='chord')[0].data

    # Obtener el valor de los BPM (tempo)
    bpm = jam.search(namespace='tempo')[0].data[0].value
    bps = bpm / 60
    # Duración de negra en segundos
    quater_duration = 1 / bps
    # Duraciones permitidas en segundos (de menor a mayor)
    duraciones_permitidas_seg = {
        'fusa': quater_duration / 8,
        'tresillo_semicorchea': quater_duration / 6,
        'fusa_puntillo': 1.5 * quater_duration / 8,
        'semicorchea': quater_duration / 4,
        'tresillo_corchea': quater_duration / 3,
        'semicorchea_puntillo': 1.5 * quater_duration / 4,
        'corchea': quater_duration / 2,
        'tresillo': (2 / 3) * quater_duration,
        'corchea_puntillo': 1.5 * quater_duration / 2,
        'negra': quater_duration,
        'negra_puntillo': quater_duration * 1.5,
        'blanca': 2 * quater_duration,
        'blanca_puntillo': 2 * quater_duration * 1.5,
        'redonda': 4 * quater_duration,
    }
    # Duración mínima permitida
    duracion_minima = duraciones_permitidas_seg['fusa']
    tresillo_minimo = quater_duration / 6
    # Definir los desfases permitidos en segundos
    #max_offset = 4 * quater_duration - duracion_minima
    offsets_permitidos_seg = [i * duracion_minima for i in range(31)]
    # Añadir múltiplos de tresillos al diccionario de desfases permitidos
    num_tresillos = 23
    for i in range(num_tresillos):
        offset_tresillo = i * tresillo_minimo
        offsets_permitidos_seg.append(offset_tresillo)
    # Ordenar los desfases permitidos
    offsets_permitidos_seg.sort()
    # Extraer todas las notas
    all_notes = [note for sublist in [list(note.data) for note in notes] for note in sublist]
    all_chords = [chord for chord in chords]
    # Ordenar las notas por tiempo
    all_notes.sort(key=lambda x: x.time)
    all_chords.sort(key=lambda x: x.time)
    # Inicializar vectores de altura, duración y desfase
    pitch_vecs = []
    dur_vecs = []
    offset_vecs = []
    start_times = []
    # Inicializar delta_t con el tiempo de inicio de la primera nota y offset inicial
    delta_t = all_notes[0].time
    tiempo_actual = 0
    # Iterar sobre las notas ordenadas
    for i in range(len(all_notes)):
        note = all_notes[i]
        pitch_vec = np.zeros(129)
        duration = note.duration
        start_time = note.time
        pitch_midi = note.value
        #encargarse de los silencios
        # Si delta_t es mayor o igual a la duración mínima permitida, agregar silencios divididos
        if delta_t >= duracion_minima:
            tiempo_actual = start_time-delta_t
            duraciones_silencio = dividir_silencio(delta_t, 4*quater_duration, duraciones_permitidas_seg)
            for duracion_silencio in duraciones_silencio:
                dur_vecs.append(duracion_a_vec(duracion_silencio, duraciones_permitidas_seg))
                current_offset = tiempo_actual % (4 * quater_duration)
                offset_vecs.append(offset_a_vec(current_offset,offsets_permitidos_seg))
                start_times.append(tiempo_actual)
                tiempo_actual += duracion_silencio
                silencio_vec = np.zeros(129)
                silencio_vec[-1] = 1  # La última componente es 1 para representar el silencio
                pitch_vecs.append(silencio_vec)
        tiempo_actual = start_time
        start_times.append(tiempo_actual)
        # Redondear pitch
        midi_note = round(pitch_midi)
        pitch_vec[midi_note] = 1
        # Añadir vector de pitch a la lista
        pitch_vecs.append(pitch_vec)
        dur_vecs.append(duracion_a_vec(duration, duraciones_permitidas_seg))
        offset_vecs.append(offset_a_vec(tiempo_actual % (4 * quater_duration), offsets_permitidos_seg))

        # Actualizar delta_t y offset para la próxima iteración
        if i < len(all_notes) - 1:
            delta_t = all_notes[i + 1].time - (start_time + duration)
    chord_vecs = []
    curr_chord_idx = 0
    for i in range(len(start_times)):
        acorde_actual = all_chords[curr_chord_idx]
        chord_time = acorde_actual.time
        chord_duration = acorde_actual.duration
        if start_times[i] > chord_time + chord_duration:
            curr_chord_idx += 1
            acorde_vec = acorde_a_vec(all_chords[curr_chord_idx])
            chord_vecs.append(acorde_vec)
        else:
            acorde_vec = acorde_a_vec(all_chords[curr_chord_idx])
            chord_vecs.append(acorde_vec)

    return pitch_vecs, dur_vecs, offset_vecs, chord_vecs
    
    

In [46]:
import tensorflow as tf
import os
data = []
ruta_directorio = '/Users/santiagogonzalez/Documents/Programación/proyectoFinalIA/data/annotation'

for nombre_archivo in os.listdir(ruta_directorio):
        if nombre_archivo.endswith('.jams'):
            ruta_archivo = os.path.join(ruta_directorio, nombre_archivo)
            try:
                pitch_vecs, dur_vecs, offset_vecs, chord_vecs = jams_a_vec(ruta_archivo)
                for i in range(len(pitch_vecs)):
                    concatenated_vector = tf.concat([pitch_vecs[i], dur_vecs[i], offset_vecs[i], chord_vecs[i]], axis=0)
                    data.append(concatenated_vector)
            except Exception as e:
                print(f"Error al convertir el archivo {ruta_archivo}: {e}")
                continue



54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
